# Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian

##### https://huggingface.co/Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian

Wav2vec2 Large 100k Vox populi настроен на русском языке с использованием Common Voice 7.0 и M-AILABS

## Подключение библиотеки

In [1]:
import os
import warnings
import pandas as pd
import re

import torch
import torchaudio

from datasets import load_dataset, load_from_disk
from datasets.features import Audio

from transformers import AutoTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor

# Для теста WER - Word Error Rate
# CER - Character Error Rate
# MER - Match Error Rate
# WIL - Word Information Lost
from jiwer import wer, cer, mer, wil

## Использование CUDA

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Get the device name
    device = torch.cuda.current_device()
    print(f"Code is connected to CUDA. Using GPU: {torch.cuda.get_device_name(device)}")
else:
    print("CUDA is not available. Running on CPU.")

In [3]:
# Все равно у меня не хватит памяти для выполнения(
device = 'cpu'

Не загружаем модель через CUDA, памяти не хватило

`RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 3.63 GiB total capacity; 2.56 GiB already allocated; 7.12 MiB free; 2.58 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF`

In [4]:
# Создадим датафрейм, который будет собирать данные и сохранять
info = pd.DataFrame()

## Загрузка модели

In [5]:
LANG_ID = "ru"
MODEL_ID = 'Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian'
PATH_MODEL = '/home/redalexdad/recognition_speech/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian'
# Кол-во текстов для предсказания
# ВНИМАНИЕ, НЕ СТАВЬ БОЛЬШЕ 10, ОС ЗАВИСНЕТ И ЯДРО УПАДЕТ!
SAMPLES = 10

In [6]:
# Проверка наличия модели в локальном пути
if os.path.exists(PATH_MODEL):
    tokenizer = AutoTokenizer.from_pretrained(PATH_MODEL)
    model = Wav2Vec2ForCTC.from_pretrained(PATH_MODEL).to(device)
    print('Успешно модель загружена')
else:
    # Загрузка токенизатора из сети
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    tokenizer.save_pretrained(PATH_MODEL)
    
    # Загрузка модели из сети
    model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(device)
    model.save_pretrained(PATH_MODEL)
    
    print(f'Успешно процессор и модель скачаны и сохранены в пути {PATH_MODEL}')

## Загрузка датасета `common_voice_11_0`

In [7]:
%%time
test_dataset_cv_11 = load_dataset("mozilla-foundation/common_voice_11_0", LANG_ID, split=f"test[:{SAMPLES}]", trust_remote_code=True)

In [8]:
resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)

In [9]:
chars_to_ignore_regex = r"[^A-Za-z0-9(),!?\'\`\"\_\n]"

In [10]:
def map_to_array(batch):
    speech, _ = torchaudio.load(batch["path"])
    batch["speech"] = resampler.forward(speech.squeeze(0)).numpy()
    batch["sampling_rate"] = resampler.new_freq
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower().replace("â€™", "'")
    
    # Токенизация и создание маски внимания
    tokens = tokenizer(batch["sentence"], return_tensors="pt", padding=True, truncation=True)
    batch["input_values"] = tokens["input_ids"]
    batch["attention_mask"] = tokens["attention_mask"]

    # Явное приведение типа данных
    batch["input_values"] = batch["input_values"].to(dtype=torch.float32)
    batch["attention_mask"] = batch["attention_mask"].to(dtype=torch.float32)
    
    return batch

### Предсказание

In [11]:
# Освобождаем памяти
torch.cuda.empty_cache()

In [12]:
def map_to_pred(batch):
    # Загрузка аудиофайла (первый элемент списка)
    audio_path = batch["path"][0]
    resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)
    speech, _ = torchaudio.load(audio_path)
    input_values = resampler.forward(speech.squeeze(0)).numpy()

    # Токенизация и получение предсказания от модели
    # добавляем размерность пакета
    input_values = torch.tensor(input_values).unsqueeze(0).to(device)  
    with torch.no_grad():
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)

    # Декодирование предсказанных токенов в текст
    transcription = tokenizer.batch_decode(predicted_ids)[0]

    return {"predicted_text": [transcription]}

In [13]:
ds = test_dataset_cv_11.map(map_to_array)

In [14]:
test_dataset_cv_11

In [15]:
result = ds.map(map_to_pred, batched=True, batch_size=1, remove_columns=list(test_dataset_cv_11.features.keys()))

### Вывод результатов

In [16]:
# Вывод оригинальных и предсказанных текстов
print('-' * 100)
for i, batch in enumerate(result["predicted_text"]):
    original_texts = test_dataset_cv_11[i]["sentence"]
    transcription = batch

    print(f"Пример {i + 1} \n- Канонический текст: {original_texts}")
    print(f"- Предсказанный текст: {transcription}")
    print("-" * 100)

### Тест WER, CER, MER, WIL

In [17]:
# Приведем все к нижнему регистру
original_texts = [ref.lower() for ref in test_dataset_cv_11["sentence"]]
transcription = [pred.lower() for pred in result["predicted_text"]]

In [18]:
# Рассчитаем WER
wer_score = wer(original_texts, transcription)

In [19]:
print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")

In [20]:
# и CER
cer_score = cer(original_texts, transcription)

In [21]:
print(f"Character Error Rate (CER): {cer_score * 100:.2f}%")

In [22]:
# MER
mer_score = mer(original_texts, transcription)

In [23]:
print(f"Match Error Rate (MER): {mer_score * 100:.2f}%")

In [24]:
# WIL
wil_score = wil(original_texts, transcription)

In [25]:
print(f"Word Information Lost (WIL): {wil_score * 100:.2f}%")

In [26]:
# Создание датафрейма
info = info.append({
    'MODEL': MODEL_ID,
    'DATASET': test_dataset_cv_11.info.dataset_name,
    'ORIGINAL TEXT': original_texts,
    'PREDICTION TEXT': transcription,
    'WER': wer_score,
    'CER': cer_score,
    'MER': mer_score,
    'WIL': wil_score,
    'SAMPLES': SAMPLES
}, ignore_index=True)
display(info)

## Загрузка датасета `common_voice_12_0`

In [27]:
%%time
test_dataset_cv_12 = load_dataset("mozilla-foundation/common_voice_12_0", LANG_ID, split=f"test[:{SAMPLES}]", trust_remote_code=True)

In [28]:
resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)

In [29]:
chars_to_ignore_regex = r"[^A-Za-z0-9(),!?\'\`\"\_\n]"

In [30]:
def map_to_array(batch):
    speech, _ = torchaudio.load(batch["path"])
    batch["speech"] = resampler.forward(speech.squeeze(0)).numpy()
    batch["sampling_rate"] = resampler.new_freq
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower().replace("â€™", "'")
    
    # Токенизация и создание маски внимания
    tokens = tokenizer(batch["sentence"], return_tensors="pt", padding=True, truncation=True)
    batch["input_values"] = tokens["input_ids"]
    batch["attention_mask"] = tokens["attention_mask"]

    # Явное приведение типа данных
    batch["input_values"] = batch["input_values"].to(dtype=torch.float32)
    batch["attention_mask"] = batch["attention_mask"].to(dtype=torch.float32)
    
    return batch

### Предсказание

In [31]:
# Освобождаем памяти
torch.cuda.empty_cache()

In [32]:
def map_to_pred(batch):
    # Загрузка аудиофайла (первый элемент списка)
    audio_path = batch["path"][0]
    resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)
    speech, _ = torchaudio.load(audio_path)
    input_values = resampler.forward(speech.squeeze(0)).numpy()

    # Токенизация и получение предсказания от модели
    # добавляем размерность пакета
    input_values = torch.tensor(input_values).unsqueeze(0).to(device)  
    with torch.no_grad():
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)

    # Декодирование предсказанных токенов в текст
    transcription = tokenizer.batch_decode(predicted_ids)[0]

    return {"predicted_text": [transcription]}

In [33]:
ds = test_dataset_cv_12.map(map_to_array)

In [34]:
test_dataset_cv_12

In [35]:
result = ds.map(map_to_pred, batched=True, batch_size=1, remove_columns=list(test_dataset_cv_12.features.keys()))

### Вывод результатов

In [36]:
# Вывод оригинальных и предсказанных текстов
print('-' * 100)
for i, batch in enumerate(result["predicted_text"]):
    original_texts = test_dataset_cv_12[i]["sentence"]
    transcription = batch

    print(f"Пример {i + 1} \n- Канонический текст: {original_texts}")
    print(f"- Предсказанный текст: {transcription}")
    print("-" * 100)

### Тест WER, CER, MER, WIL

In [37]:
# Приведем все к нижнему регистру
original_texts = [ref.lower() for ref in test_dataset_cv_12["sentence"]]
transcription = [pred.lower() for pred in result["predicted_text"]]

In [38]:
# Рассчитаем WER
wer_score = wer(original_texts, transcription)

In [39]:
print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")

In [40]:
# и CER
cer_score = cer(original_texts, transcription)

In [41]:
print(f"Character Error Rate (CER): {cer_score * 100:.2f}%")

In [42]:
# MER
mer_score = mer(original_texts, transcription)

In [43]:
print(f"Match Error Rate (MER): {mer_score * 100:.2f}%")

In [44]:
# WIL
wil_score = wil(original_texts, transcription)

In [45]:
print(f"Word Information Lost (WIL): {wil_score * 100:.2f}%")

In [46]:
# Создание датафрейма
info = info.append({
    'MODEL': MODEL_ID,
    'DATASET': test_dataset_cv_12.info.dataset_name,
    'ORIGINAL TEXT': original_texts,
    'PREDICTION TEXT': transcription,
    'WER': wer_score,
    'CER': cer_score,
    'MER': mer_score,
    'WIL': wil_score,
    'SAMPLES': SAMPLES
}, ignore_index=True)
display(info)

## Загрузка датасета `common_voice_14_0`

In [47]:
%%time
test_dataset_cv_14 = load_dataset("mozilla-foundation/common_voice_14_0", LANG_ID, split=f"test[:{SAMPLES}]", trust_remote_code=True)

In [48]:
resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)

In [49]:
chars_to_ignore_regex = r"[^A-Za-z0-9(),!?\'\`\"\_\n]"

In [50]:
def map_to_array(batch):
    speech, _ = torchaudio.load(batch["path"])
    batch["speech"] = resampler.forward(speech.squeeze(0)).numpy()
    batch["sampling_rate"] = resampler.new_freq
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower().replace("â€™", "'")
    
    # Токенизация и создание маски внимания
    tokens = tokenizer(batch["sentence"], return_tensors="pt", padding=True, truncation=True)
    batch["input_values"] = tokens["input_ids"]
    batch["attention_mask"] = tokens["attention_mask"]

    # Явное приведение типа данных
    batch["input_values"] = batch["input_values"].to(dtype=torch.float32)
    batch["attention_mask"] = batch["attention_mask"].to(dtype=torch.float32)
    
    return batch

### Предсказание

In [51]:
# Освобождаем памяти
torch.cuda.empty_cache()

In [52]:
def map_to_pred(batch):
    # Загрузка аудиофайла (первый элемент списка)
    audio_path = batch["path"][0]
    resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)
    speech, _ = torchaudio.load(audio_path)
    input_values = resampler.forward(speech.squeeze(0)).numpy()

    # Токенизация и получение предсказания от модели
    # добавляем размерность пакета
    input_values = torch.tensor(input_values).unsqueeze(0).to(device)  
    with torch.no_grad():
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)

    # Декодирование предсказанных токенов в текст
    transcription = tokenizer.batch_decode(predicted_ids)[0]

    return {"predicted_text": [transcription]}

In [53]:
ds = test_dataset_cv_14.map(map_to_array)

In [54]:
test_dataset_cv_14

In [55]:
result = ds.map(map_to_pred, batched=True, batch_size=1, remove_columns=list(test_dataset_cv_14.features.keys()))

### Вывод результатов

In [56]:
# Вывод оригинальных и предсказанных текстов
print('-' * 100)
for i, batch in enumerate(result["predicted_text"]):
    original_texts = test_dataset_cv_14[i]["sentence"]
    transcription = batch

    print(f"Пример {i + 1} \n- Канонический текст: {original_texts}")
    print(f"- Предсказанный текст: {transcription}")
    print("-" * 100)

### Тест WER, CER, MER, WIL

In [57]:
# Приведем все к нижнему регистру
original_texts = [ref.lower() for ref in test_dataset_cv_14["sentence"]]
transcription = [pred.lower() for pred in result["predicted_text"]]

In [58]:
# Рассчитаем WER
wer_score = wer(original_texts, transcription)

In [59]:
print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")

In [60]:
# и CER
cer_score = cer(original_texts, transcription)

In [61]:
print(f"Character Error Rate (CER): {cer_score * 100:.2f}%")

In [62]:
# MER
mer_score = mer(original_texts, transcription)

In [63]:
print(f"Match Error Rate (MER): {mer_score * 100:.2f}%")

In [64]:
# WIL
wil_score = wil(original_texts, transcription)

In [65]:
print(f"Word Information Lost (WIL): {wil_score * 100:.2f}%")

In [66]:
# Создание датафрейма
info = info.append({
    'MODEL': MODEL_ID,
    'DATASET': test_dataset_cv_14.info.dataset_name,
    'ORIGINAL TEXT': original_texts,
    'PREDICTION TEXT': transcription,
    'WER': wer_score,
    'CER': cer_score,
    'MER': mer_score,
    'WIL': wil_score,
    'SAMPLES': SAMPLES
}, ignore_index=True)
display(info)

In [67]:
info.to_csv('wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian.csv', index=False)